In [1]:
import pandas as pd

# Load the datasets
train_data = pd.read_csv("C:/Users/harsharaj/Downloads/archive (1)/udupi train.csv")
test_data = pd.read_csv("C:/Users/harsharaj/Downloads/archive (1)/udupi test.csv")

# Display the first few rows of the training data
train_data.head(), train_data.describe(), train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          24 non-null     object 
 1   meantemp      24 non-null     float64
 2   humidity      24 non-null     int64  
 3   meanpressure  24 non-null     float64
 4   wind_speed    24 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.1+ KB


(      date  meantemp  humidity  meanpressure  wind_speed
 0  12:28.4      27.0        79        1013.0        10.0
 1  12:28.4      26.0        77        1013.0        10.0
 2  12:28.4      25.0        75        1013.0        10.0
 3  12:28.4      27.0        79        1013.5        10.8
 4  12:28.4      28.0        76        1013.5        10.8,
         meantemp   humidity  meanpressure  wind_speed
 count  24.000000  24.000000     24.000000   24.000000
 mean   26.812500  76.750000   1013.458333   10.408333
 std     1.405057   1.452135      0.387766    0.346306
 min    25.000000  75.000000   1013.000000   10.000000
 25%    25.500000  75.750000   1013.000000   10.150000
 50%    26.750000  76.500000   1013.500000   10.400000
 75%    28.000000  78.000000   1013.625000   10.800000
 max    29.500000  79.000000   1014.000000   11.000000,
 None)

In [2]:
# Convert the 'date' column to datetime
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

# Identify and remove outliers in 'meanpressure' (assuming values > 2000 are outliers)
train_data = train_data[train_data['meanpressure'] <= 2000]

# Define features (X) and target (y)
X_train = train_data[['meantemp', 'humidity', 'wind_speed', 'meanpressure']]
y_train = train_data['meantemp']

X_test = test_data[['meantemp', 'humidity', 'wind_speed', 'meanpressure']]
y_test = test_data['meantemp']

# Check the updated datasets
train_data.describe(), test_data.describe()


C:\Users\harsharaj\AppData\Local\Temp\ipykernel_23960\1388877805.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data['date'] = pd.to_datetime(train_data['date'])
C:\Users\harsharaj\AppData\Local\Temp\ipykernel_23960\1388877805.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test_data['date'] = pd.to_datetime(test_data['date'])


(                      date   meantemp   humidity  meanpressure  wind_speed
 count                   24  24.000000  24.000000     24.000000   24.000000
 mean   2024-07-11 12:28:24  26.812500  76.750000   1013.458333   10.408333
 min    2024-07-11 12:28:24  25.000000  75.000000   1013.000000   10.000000
 25%    2024-07-11 12:28:24  25.500000  75.750000   1013.000000   10.150000
 50%    2024-07-11 12:28:24  26.750000  76.500000   1013.500000   10.400000
 75%    2024-07-11 12:28:24  28.000000  78.000000   1013.625000   10.800000
 max    2024-07-11 12:28:24  29.500000  79.000000   1014.000000   11.000000
 std                    NaN   1.405057   1.452135      0.387766    0.346306,
                       date   meantemp   humidity  meanpressure  wind_speed
 count                    7   7.000000   7.000000      7.000000    7.000000
 mean   2024-07-11 12:28:24  28.428571  77.571429   1013.571429   10.742857
 min    2024-07-11 12:28:24  26.500000  75.000000   1013.000000   10.400000
 25%    202

In [5]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.metrics import mean_squared_error, r2_score

# Load the datasets
train_data = pd.read_csv("C:/Users/harsharaj/Downloads/archive (1)/udupi train.csv")
test_data = pd.read_csv("C:/Users/harsharaj/Downloads/archive (1)/udupi test.csv")

# Convert the 'date' column to datetime
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

# Identify and remove outliers in 'meanpressure' (assuming values > 2000 are outliers)
train_data = train_data[train_data['meanpressure'] <= 2000]

# Define features (X) and target (y)
X_train = train_data[['meantemp', 'humidity', 'wind_speed', 'meanpressure']]
y_train = train_data['meantemp']

X_test = test_data[['meantemp', 'humidity', 'wind_speed', 'meanpressure']]
y_test = test_data['meantemp']

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Discretize the target variable into 10 bins
kbins = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
y_train_binned = kbins.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_binned = kbins.transform(y_test.values.reshape(-1, 1)).ravel()

# Train the Gaussian Naive Bayes model
gnb_model = GaussianNB()
gnb_model.fit(X_train_scaled, y_train_binned)

# Predict on the test data
y_pred_gnb_binned = gnb_model.predict(X_test_scaled)

# Calculate the accuracy metrics
mse_gnb_binned = mean_squared_error(y_test_binned, y_pred_gnb_binned)
r2_gnb_binned = r2_score(y_test_binned, y_pred_gnb_binned)

print("Linear regression accuracy",mse_gnb_binned,"\nNaive bias accuracy", r2_gnb_binned,"\nUsing ensemble method accuracy",(mse_gnb_binned + r2_gnb_binned)/2)



Linear regression accuracy 0.5714285714285714 
Naive bias accuracy 0.8613861386138614 
Using ensemble method accuracy 0.7164073550212164


C:\Users\harsharaj\AppData\Local\Temp\ipykernel_23960\1709038512.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data['date'] = pd.to_datetime(train_data['date'])
C:\Users\harsharaj\AppData\Local\Temp\ipykernel_23960\1709038512.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test_data['date'] = pd.to_datetime(test_data['date'])


In [7]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.metrics import mean_squared_error, r2_score

# Load the datasets
train_data = pd.read_csv("C:/Users/harsharaj/Downloads/archive (1)/udupi train.csv")
test_data = pd.read_csv("C:/Users/harsharaj/Downloads/archive (1)/udupi test.csv")

# Convert the 'date' column to datetime
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

# Identify and remove outliers in 'meanpressure' (assuming values > 2000 are outliers)
train_data = train_data[train_data['meanpressure'] <= 2000]

# Define features (X) and target (y)
X_train = train_data[['meantemp', 'humidity', 'wind_speed', 'meanpressure']]
y_train = train_data['meantemp']

X_test = test_data[['meantemp', 'humidity', 'wind_speed', 'meanpressure']]
y_test = test_data['meantemp']

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Discretize the target variable into 10 bins
kbins = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
y_train_binned = kbins.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_binned = kbins.transform(y_test.values.reshape(-1, 1)).ravel()

# Train the Linear Regression model
linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train, y_train)

# Train the Gaussian Naive Bayes model
gnb_model = GaussianNB()
gnb_model.fit(X_train_scaled, y_train_binned)

# Function to predict next day's mean temperature
def predict_next_day(temp, humidity, wind_speed, pressure):
    input_data = np.array([[temp, humidity, wind_speed, pressure]])
    input_data_scaled = scaler.transform(input_data)
    
    # Linear Regression prediction
    pred_lr = linear_reg_model.predict(input_data)[0]
    
    # Naive Bayes prediction
    pred_gnb_binned = gnb_model.predict(input_data_scaled)[0]
    pred_gnb = kbins.inverse_transform([[pred_gnb_binned]])[0][0]
    
    return pred_lr, pred_gnb

# Example input values of today
next_day_temp = 26.0 
next_day_humidity = 91.0  
next_day_wind_speed = 8.0  
next_day_pressure = 1000.0  

# Predict the next day's mean temperature
predicted_temp_lr, predicted_temp_gnb = predict_next_day(next_day_temp, next_day_humidity, next_day_wind_speed, next_day_pressure)
ensemble=(predicted_temp_lr +predicted_temp_gnb)/2
print("Linear regression temp",predicted_temp_lr,"Naive bias regression temp" ,predicted_temp_gnb)
print("using ensemble method temp",ensemble)


Linear regression temp 25.999999999999996 Naive bias regression temp 27.025
using ensemble method temp 26.512499999999996


C:\Users\harsharaj\AppData\Local\Temp\ipykernel_23960\557749732.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data['date'] = pd.to_datetime(train_data['date'])
C:\Users\harsharaj\AppData\Local\Temp\ipykernel_23960\557749732.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test_data['date'] = pd.to_datetime(test_data['date'])
c:\Users\harsharaj\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\harsharaj\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
